In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

# Pergunta ao usuário qual produto deseja procurar
produto = input('Qual produto você deseja procurar na Boticário: ')

# Configuração do WebDriver
navegador = webdriver.Chrome()

# Maximiza a janela
navegador.maximize_window()

# URL da busca com o produto
url = f'https://www.boticario.com.br/busca?q={produto}'

# Abre a URL no navegador
navegador.get(url)

# Aceita cookies se necessário
try:
    wait = WebDriverWait(navegador, 10)
    botao_aceitar_cookies = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
    botao_aceitar_cookies.click()
except Exception as e:
    print('Não foi possível aceitar os cookies.')

# Encontra todos os elementos de classe 'showcase-image'
elementos_produtos = navegador.find_elements(By.CLASS_NAME, 'showcase-image')

# Lista para armazenar os resultados
resultados = []

# Itera sobre os elementos para obter informações
for i in range(len(elementos_produtos)):
    try:
        elemento = elementos_produtos[i]
        url_imagem = elemento.get_attribute('src')

        # Clicar na imagem para abrir detalhes do produto
        elemento.click()

        # Aguarda a visibilidade do título (h1)
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'h1')))
        elemento_h1 = navegador.find_element(By.TAG_NAME, 'h1')
        titulo_h1 = elemento_h1.text

        # Aguarda a visibilidade do preço
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'nproduct-price-value')))
        elemento_preco = navegador.find_element(By.CLASS_NAME, 'nproduct-price-value')
        preco_texto = elemento_preco.text.replace('R$', '').replace(',', '.').strip()

        # Converte para float e depois para inteiro
        preco = int(float(preco_texto))

        # Adiciona os resultados à lista
        resultados.append({
            'titulo': titulo_h1,
            'preco': preco,
            'avaliacao': i  # Adicione a avaliação real, se disponível
        })

        # Voltar para a página de resultados
        navegador.back()

        # Aguarde a visibilidade dos elementos novamente
        wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'showcase-image')))
        elementos_produtos = navegador.find_elements(By.CLASS_NAME, 'showcase-image')

    except IndexError:
        print(f"Erro: Não foi possível acessar o índice {i} na lista. Provavelmente, não há mais elementos.")
        break

indices = []
titulos = []
precos = []
avaliacoes = []

# Loop para iterar sobre os resultados
for idx, resultado in enumerate(resultados, start=1):
    # Seu código para extrair os dados...

    # Exemplo: Se 'resultado' for um dicionário, você pode acessar as chaves assim:
    titulo = resultado.get('titulo', 'N/A')  # 'N/A' é um valor padrão caso 'titulo' não esteja presente
    preco = resultado.get('preco', 'N/A')  # Substitua 'preco' pela chave real
    avaliacao = resultado.get('avaliacao', 'N/A')  # Substitua 'avaliacao' pela chave real

    # Adicionar dados às listas
    indices.append(idx)
    titulos.append(titulo)
    precos.append(preco)
    avaliacoes.append(avaliacao)

    # Exibir os dados
    print(f'{idx}: {titulo}, {preco}, {avaliacao}')

# Criar DataFrame a partir das listas
tabela = pd.DataFrame({
    'Índice': indices,
    'Título': titulos,
    'Preço': precos,
    'Avaliação': avaliacoes
})

# Exibir o DataFrame
print(tabela)

data_atual = datetime.now().strftime("%Y-%m-%d")

nome_arquivo = f'{produto}_resultados_{data_atual}.xlsx'

# Salvar o DataFrame em um arquivo Excel
tabela.to_excel(nome_arquivo, index=False)

navegador.quit()

1: Sabonete Líquido Corporal Nativa SPA Morango Ruby 200ml, 31, 0
2: Sabonete em Barra Cuide-se Bem Boa Noite 2 Unidades 80g Cada, 21, 1
3: Sabonete em Barra Cuide-se Bem Nuvem 5 unidades de 80g cada, 34, 2
4: Sabonete em Barra Duo Cuide-se Bem Rosa e Algodão, 2 unidades de 80g cada, 21, 3
5: Combo Presente Cuide-se Bem Beijinho: 2 Sabonetes em Barra + Creme para Mãos 50g + Caixa de Presente, 54, 4
6: Combo Cuide-Se Bem Beijinho: Creme Para Mãos, 50G + Sabonete Em Barra, 2 Un. 80G Cada, 54, 5
7: Combo Cuide-Se Bem Beijinho: Loção Hidratante, 400Ml + Sabonete Em Barra, 2 Un. 80G Cada, 84, 6
8: Sabonete Em Barra Quinteto Cuide-Se Bem Deleite, 5 Unidades De 80 g Cada, 34, 7
9: Combo Presente Nativa SPA Ameixa: Sabonete em Barra 90g + Creme Para as Mãos 75g, 56, 8
10: Sabonete Em Barra Cuide-Se Bem Beijinho, 2 Unidades De 80 g Cada, 21, 9
11: Sabonete em Barra Cuide-se Bem Boa Noite 4 Unidades 80g Cada, 34, 10
12: Sabonete Líquido Esfoliante Corporal Anti-stress Nativa SPA Jasmim Sambac 17